# Fission Sources, Particle Tracks, and Fixed Source Problems

In [1]:
import openmc
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

## Fission Sources

Here we'll discuss visualization of OpenMC's fission sources and defining source distributions for fixed source problems.

Fission sources iterated during the inactive cycles of an eigenvalue solution. They are written as part of OpenMC statepoint files. Here, we'll look at how to examine and vizualize the fission source.

First, we'll need a model to work with.

In [2]:
model = openmc.examples.pwr_pin_cell()
# extend model to 3D
z_min = openmc.ZPlane(-50.0, boundary_type='vacuum')
z_max = openmc.ZPlane(50.0, boundary_type='vacuum')

model_cells = model.geometry.get_all_cells()
for cell in model_cells.values():
    cell.region &= +z_min & -z_max

# Working with track files

OpenMC can generate track files for particles, including secondary particles. A track "state" is stored at each collision and includes the following information:

r, u, E, time, wgt, cell_id, cell_instance, and material_id

  - Position in Cartesian coordinates, `r`
  - Unit vector for the particle's direction, `u`
  - Energy, `E`
  - Time, `t`
  - Cell ID, `cell_id`
  - Cell Instance, `cell_instance`
  - Material ID, `material_id`
  
We're going to tell the next OpenMC run to write out the first 100 particles by applying the following settings:

In [ ]:
model.settings.max_tracks = 5000
model.run(tracks=True)

## Fixed Source Problems

Unlike eigenvalue problems in which inactive batches are necessary for the source to converge, fixed source problems denote a class of problems in which the exact source is specified as part of the problem setup.

First we'll create a problem to work with. We'll start by looking at a simple planar shield with a point source on one side.

In [20]:
# define some shielding materials
water = openmc.Material(name='Water')
water.set_density('g/cc', 1.0)
water.add_element('H', 2.0)
water.add_element('O', 1.0)

steel = openmc.Material(name='Steel')
steel.set_density('g/cc', 8.0)
steel.add_element('Si', 0.010048)
steel.add_element('S', 0.00023)
steel.add_element('Fe', 0.669)
steel.add_element('Ni', 0.12)
steel.add_element('Mo', 0.025)
steel.add_nuclide('P31',0.00023)
steel.add_nuclide('Mn55',0.011014)

air = openmc.Material(name='Air')
air.set_density('g/cc', 0.001205)
air.add_element('N', 0.784431)
air.add_element('O', 0.210748)
air.add_element('Ar',0.0046)

iron = openmc.Material(name="Iron")
iron.set_density("g/cm3", 7.874)
iron.add_nuclide("Fe54", 0.0564555822608)
iron.add_nuclide("Fe56", 0.919015287728)
iron.add_nuclide("Fe57", 0.0216036861685)
iron.add_nuclide("Fe58", 0.00292544384231)

material_colors = {air: 'white',
                   steel: 'gray',
                   water: 'blue',
                   iron: 'brown'}

In [21]:
# create a geometry
x0 = openmc.XPlane(0.0, boundary_type='vacuum')
x1 = openmc.XPlane(5.0)
x2 = openmc.XPlane(8.0)
x3 = openmc.XPlane(10.0)
x4 = openmc.XPlane(12.0)
x5 = openmc.XPlane(15.0, boundary_type='vacuum')

# can also use this convenience function
# regions = openmc.model.subdivide([x0, x1, x2, x3, x4])

y_min = openmc.YPlane(-5.0, boundary_type='vacuum')
y_max = openmc.YPlane(5.0, boundary_type='vacuum')
z_min = openmc.ZPlane(-5.0, boundary_type='vacuum')
z_max = openmc.ZPlane(5.0, boundary_type='vacuum')

In [22]:
cell0 = openmc.Cell(fill=air, region=+x0 & -x1)
cell1 = openmc.Cell(fill=steel, region=+x1 & -x2)
cell2 = openmc.Cell(fill=water, region=+x2 & -x3)
cell3 = openmc.Cell(fill=iron, region=+x3 & -x4)
cell4 = openmc.Cell(fill=air, region=+x4 & -x5)

cells = [cell0, cell1, cell2, cell3, cell4]

In [23]:
for cell in cells:
    cell.region &= +y_min & -y_max & +z_min & -z_max

In [24]:
geometry = openmc.Geometry(cells)
model = openmc.Model(geometry=geometry)

## Defining a source term

Now we'll define a source. To start, we'll define a simple point source.

Sources in OpenMC are composed of space, angle, and energy distributions. There are a number of distributions available in the `openmc.stats` module to helps us define these sources.

## Line Sources

Now let's update our source term a bit. We'll define a line source parallel to the Y axis.

## Discrete Sources

## Cylindrical Sources

# Volume sources

A source can also be specified using a domain which can either be an iterable of cells, universes, regions, geometries or materials